# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of a cell

In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, Vector2, Vector3, Quaternion
be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset()

In [ ]:
def add_cell(position_random_seed, orientation_randon_seed):
    
    core.set_application_parameters(image_stream_fps=0)
    representation=be.REPRESENTATION_ATOMS_AND_STICKS

    # Resources
    resource_folder = '../../tests/test_files/'
    pdb_folder = resource_folder + 'pdb/'
    membrane_folder = pdb_folder + 'membrane/'

    # ACE2 receptor definition
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'],
        occurences=8,
        position=Vector3(0.0, 6.0, 0.0),
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0)
    )

    # Membrane definition
    membrane_size = 800.0
    membrane_amplitude = 80.0 # For sinusoidal shape only
    membrane = Membrane(
        sources=[
            membrane_folder + 'segA.pdb',
            membrane_folder + 'segB.pdb',
            membrane_folder + 'segC.pdb',
            membrane_folder + 'segD.pdb'
        ],
        occurences=200000)

    # Cell definition
    cell = Cell(
        name='Cell',size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        membrane=membrane, receptor=ace2_receptor,
        random_position_strength=2.0,
        random_position_seed=position_random_seed,
        random_rotation_strength=0.5,
        random_rotation_seed=orientation_randon_seed,
        extra_parameters=[membrane_amplitude]
    )

    # Add cell to scene
    be.add_cell(cell=cell, representation=representation)
    
    # Materials
    be.apply_default_color_scheme(be.SHADING_MODE_DIFFUSE)
    
    be.set_models_visibility(True)
    core.set_application_parameters(image_stream_fps=20)

In [ ]:
add_cell(0, 0)

In [ ]:
status = core.set_camera(
    orientation=[-0.085, 0.340, 0.022, 0.936],
    position=[461.372, 55.791, 480.118],
    target=[23.322, -65.160, -40.892]
)

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
from mediamaker import MovieMaker
from IPython.display import clear_output

mm = MovieMaker(be)
output_folder = '/tmp'

for frame in range(1, 20):
    clear_output()
    print('Frame %i' % frame)

    '''Cell'''
    add_cell(frame + 1, frame + 2)
    
    '''Snapshot'''
    mm.create_snapshot(
        size=[256, 256], samples_per_pixel=64,
        path=output_folder + '/%05d.png' % frame)